# 01 - Process Rosbags for Training Data

**RAPP Lab 04 - Vision-Action Model**

This notebook processes rosbag recordings to extract and prepare training data:

1. **Extract Topics:** Load skeleton tracking and joint state data
2. **Synchronize:** Align messages by timestamp (interpolate joint states to skeleton rate)
3. **Interactive Selection:** Choose which skeleton is the "leader" with 3D visualization
4. **Transform:** Convert skeleton to robot_base_link coordinate frame
5. **Quality Checks:** Validate tracking confidence, detect position jumps, check joint limits
6. **Export CSV:** Save synchronized data for training
7. **Update Metadata:** Track processed recordings

---

**MANUAL CONFIGURATION:** Set the rosbag name below for each processing run.

In [ ]:
# =============================================================================
# MANUAL CONFIGURATION - Change this for each rosbag
# =============================================================================

ROSBAG_NAME = '25_12_11_RAPP_M_R2G1S1_01'  # <-- CHANGE THIS FOR EACH ROSBAG

# =============================================================================
# Do not modify below this line
# =============================================================================

## 1. Imports and Setup

In [ ]:
import os
import sys
from pathlib import Path

# Ensure vam_utils is importable
workspace_dir = Path('/workspace')
if str(workspace_dir) not in sys.path:
    sys.path.insert(0, str(workspace_dir))

import numpy as np
import pandas as pd
import sqlite3
from typing import Optional, Tuple, List, Dict, Any
from datetime import datetime
import yaml

# ROS2 message deserialization
from rclpy.serialization import deserialize_message
from sensor_msgs.msg import JointState
from builtin_interfaces.msg import Time

# For skeleton messages - we'll need to handle this
import importlib.util

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load URDF for FK
from urdf_parser_py.urdf import URDF

print("Imports successful!")

In [ ]:
# Configuration paths
ROSBAG_DIR = Path('/data/rosbags')
PROCESSED_DIR = Path('/data/processed')
CONFIG_DIR = Path('/config')

# Construct full path to rosbag database
rosbag_path = ROSBAG_DIR / ROSBAG_NAME
db_file = list(rosbag_path.glob('*.db3'))[0] if rosbag_path.exists() else None

# Output paths
csv_output_dir = PROCESSED_DIR / 'csv'
csv_output_dir.mkdir(parents=True, exist_ok=True)

metadata_file = PROCESSED_DIR / 'recordings_metadata.csv'

print(f"Rosbag directory: {rosbag_path}")
print(f"Database file: {db_file}")
print(f"CSV output directory: {csv_output_dir}")
print(f"Metadata file: {metadata_file}")

# Verify paths
assert rosbag_path.exists(), f"Rosbag directory not found: {rosbag_path}"
assert db_file is not None and db_file.exists(), f"No .db3 file found in {rosbag_path}"

## 2. Helper Functions

In [ ]:
def get_topic_messages_with_timestamps(db_path: Path, topic_name: str) -> List[Tuple[int, bytes]]:
    """
    Extract raw serialized messages with timestamps from a rosbag database.
    
    Args:
        db_path: Path to the .db3 file
        topic_name: The ROS topic to extract
        
    Returns:
        List of (timestamp_ns, serialized_message) tuples
    """
    conn = sqlite3.connect(str(db_path))
    cursor = conn.cursor()
    
    # Get topic ID
    cursor.execute("SELECT id FROM topics WHERE name = ?", (topic_name,))
    result = cursor.fetchone()
    if result is None:
        conn.close()
        raise ValueError(f"Topic '{topic_name}' not found in rosbag")
    
    topic_id = result[0]
    
    # Get messages with timestamps
    cursor.execute(
        "SELECT timestamp, data FROM messages WHERE topic_id = ? ORDER BY timestamp",
        (topic_id,)
    )
    messages = cursor.fetchall()
    
    conn.close()
    return messages


def list_topics(db_path: Path) -> Dict[str, Dict[str, Any]]:
    """List all topics in a rosbag with their message counts and types."""
    conn = sqlite3.connect(str(db_path))
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT t.name, t.type, COUNT(m.id) as msg_count
        FROM topics t
        LEFT JOIN messages m ON t.id = m.topic_id
        GROUP BY t.id
    """)
    
    topics = {}
    for name, msg_type, count in cursor.fetchall():
        topics[name] = {'type': msg_type, 'count': count}
    
    conn.close()
    return topics


def timestamp_to_seconds(timestamp_ns: int) -> float:
    """Convert nanosecond timestamp to seconds (float)."""
    return timestamp_ns / 1e9


# List available topics
topics = list_topics(db_file)
print("\nAvailable topics:")
for name, info in sorted(topics.items()):
    print(f"  {name}: {info['type']} ({info['count']} messages)")

## 3. Load URDF and Forward Kinematics

Reuse the forward kinematics implementation from Phase 1.

In [ ]:
# Load URDF
urdf_path = PROCESSED_DIR / 'ur10.urdf'
assert urdf_path.exists(), f"URDF not found at {urdf_path}. Run 00_setup_urdf.ipynb first."

with open(urdf_path, 'r') as f:
    urdf_string = f.read()

robot = URDF.from_xml_string(urdf_string)

print(f"Loaded URDF: {robot.name}")
print(f"Joints: {len([j for j in robot.joints if j.type in ['revolute', 'continuous']])}")

In [ ]:
# Copy FK class from Phase 1 (simplified version)

def quaternion_to_rotation_matrix(q: np.ndarray) -> np.ndarray:
    """Convert quaternion (xyzw) to 3x3 rotation matrix."""
    x, y, z, w = q
    norm = np.sqrt(x*x + y*y + z*z + w*w)
    x, y, z, w = x/norm, y/norm, z/norm, w/norm
    
    return np.array([
        [1 - 2*(y*y + z*z), 2*(x*y - z*w), 2*(x*z + y*w)],
        [2*(x*y + z*w), 1 - 2*(x*x + z*z), 2*(y*z - x*w)],
        [2*(x*z - y*w), 2*(y*z + x*w), 1 - 2*(x*x + y*y)]
    ])


def rpy_to_rotation_matrix(rpy: Tuple[float, float, float]) -> np.ndarray:
    """Convert roll-pitch-yaw angles to 3x3 rotation matrix."""
    roll, pitch, yaw = rpy
    cr, sr = np.cos(roll), np.sin(roll)
    cp, sp = np.cos(pitch), np.sin(pitch)
    cy, sy = np.cos(yaw), np.sin(yaw)
    
    return np.array([
        [cy*cp, cy*sp*sr - sy*cr, cy*sp*cr + sy*sr],
        [sy*cp, sy*sp*sr + cy*cr, sy*sp*cr - cy*sr],
        [-sp, cp*sr, cp*cr]
    ])


def rotation_matrix_from_axis_angle(axis: np.ndarray, angle: float) -> np.ndarray:
    """Create rotation matrix from axis-angle representation."""
    axis = np.array(axis) / np.linalg.norm(axis)
    K = np.array([
        [0, -axis[2], axis[1]],
        [axis[2], 0, -axis[0]],
        [-axis[1], axis[0], 0]
    ])
    return np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * (K @ K)


class UR10ForwardKinematics:
    """Forward kinematics for UR10 robot."""
    
    def __init__(self, urdf: URDF):
        self.urdf = urdf
        self.joint_names = []
        self.joint_info = {}
        
        for joint in urdf.joints:
            if joint.type in ['revolute', 'continuous']:
                self.joint_names.append(joint.name)
                self.joint_info[joint.name] = {
                    'parent': joint.parent,
                    'child': joint.child,
                    'origin_xyz': joint.origin.xyz if joint.origin else [0, 0, 0],
                    'origin_rpy': joint.origin.rpy if joint.origin else [0, 0, 0],
                    'axis': joint.axis if joint.axis else [0, 0, 1],
                    'limits': (joint.limit.lower, joint.limit.upper) if joint.limit else (-np.pi, np.pi)
                }
    
    def compute_transforms(self, joint_angles: Dict[str, float]) -> Dict[str, np.ndarray]:
        """Compute 4x4 homogeneous transforms for all links."""
        transforms = {}
        base_link = self.urdf.get_root()
        transforms[base_link] = np.eye(4)
        
        for joint in self.urdf.joints:
            parent = joint.parent
            child = joint.child
            
            if parent not in transforms:
                continue
            
            T = np.eye(4)
            if joint.origin:
                T[:3, 3] = joint.origin.xyz
                T[:3, :3] = rpy_to_rotation_matrix(joint.origin.rpy)
            
            if joint.type in ['revolute', 'continuous']:
                angle = joint_angles.get(joint.name, 0.0)
                axis = joint.axis if joint.axis else [0, 0, 1]
                R_joint = rotation_matrix_from_axis_angle(axis, angle)
                T[:3, :3] = T[:3, :3] @ R_joint
            
            transforms[child] = transforms[parent] @ T
        
        return transforms
    
    def get_joint_positions(self, joint_angles: Dict[str, float]) -> List[np.ndarray]:
        """Get positions of joint origins."""
        transforms = self.compute_transforms(joint_angles)
        positions = []
        
        base = self.urdf.get_root()
        positions.append(transforms[base][:3, 3])
        
        for joint_name in self.joint_names:
            child = self.joint_info[joint_name]['child']
            if child in transforms:
                positions.append(transforms[child][:3, 3])
        
        return positions


fk = UR10ForwardKinematics(robot)
print(f"\nInitialized FK with {len(fk.joint_names)} joints")
print(f"Joint order: {fk.joint_names}")

## 4. Extract Joint States

In [ ]:
# Extract joint state messages with timestamps
print("Extracting joint states...")
joint_state_data = get_topic_messages_with_timestamps(db_file, '/joint_states')

print(f"Found {len(joint_state_data)} joint state messages")

# Parse joint states into DataFrame
joint_records = []
for timestamp_ns, msg_bytes in joint_state_data:
    msg = deserialize_message(msg_bytes, JointState)
    
    # Create joint angle dictionary
    joint_angles = {name: pos for name, pos in zip(msg.name, msg.position)}
    
    # Map to URDF joint names in order
    ordered_angles = []
    for urdf_joint_name in fk.joint_names:
        # Try direct match or partial match
        angle = None
        for js_name, js_angle in joint_angles.items():
            if urdf_joint_name == js_name or urdf_joint_name in js_name or js_name in urdf_joint_name:
                angle = js_angle
                break
        ordered_angles.append(angle if angle is not None else 0.0)
    
    joint_records.append({
        'timestamp': timestamp_to_seconds(timestamp_ns),
        'j0': ordered_angles[0],
        'j1': ordered_angles[1],
        'j2': ordered_angles[2],
        'j3': ordered_angles[3],
        'j4': ordered_angles[4],
        'j5': ordered_angles[5]
    })

df_joints = pd.DataFrame(joint_records)
print(f"\nJoint states DataFrame shape: {df_joints.shape}")
print(f"Time range: {df_joints['timestamp'].min():.2f}s to {df_joints['timestamp'].max():.2f}s")
print(f"Duration: {df_joints['timestamp'].max() - df_joints['timestamp'].min():.2f}s")
print(f"Average frequency: {len(df_joints) / (df_joints['timestamp'].max() - df_joints['timestamp'].min()):.1f} Hz")

df_joints.head()

## 5. Extract Skeleton Data

**Note:** We need to handle ZED skeleton messages. Since `zed_msgs` might not be directly available,
we'll parse the raw message data. The skeleton message contains an array of detected skeletons,
each with 16 keypoints (x, y, z) and confidence scores.

In [ ]:
# Try to import zed_msgs, if not available we'll parse manually
try:
    from zed_msgs.msg import ObjectsStamped
    ZED_MSGS_AVAILABLE = True
    print("zed_msgs available")
except ImportError:
    ZED_MSGS_AVAILABLE = False
    print("Warning: zed_msgs not available, will attempt manual parsing")
    print("Skeleton extraction may be limited")

# Extract skeleton messages
print("\nExtracting skeleton data...")
skeleton_data = get_topic_messages_with_timestamps(db_file, '/zed/zed_node/body_trk/skeletons')
print(f"Found {len(skeleton_data)} skeleton messages")

In [ ]:
# Parse skeleton data
skeleton_records = []

if ZED_MSGS_AVAILABLE:
    for timestamp_ns, msg_bytes in skeleton_data:
        msg = deserialize_message(msg_bytes, ObjectsStamped)
        
        # Process each detected skeleton
        for skeleton_obj in msg.objects:
            # Extract 16 keypoints (ZED body tracking 16-point mode)
            keypoints = skeleton_obj.skeleton_2d.keypoints  # or skeleton_3d depending on what's available
            
            # Note: We want 3D positions - check if skeleton_3d is available
            if hasattr(skeleton_obj, 'skeleton_3d'):
                keypoints = skeleton_obj.skeleton_3d.keypoints
            
            # Build record
            record = {
                'timestamp': timestamp_to_seconds(timestamp_ns),
                'skeleton_id': skeleton_obj.label_id  # 0 or 1 typically
            }
            
            # Add all 16 keypoints (x, y, z)
            for i, kp in enumerate(keypoints[:16]):  # Ensure only 16 keypoints
                record[f'sk_{i}_x'] = kp.kp[0]
                record[f'sk_{i}_y'] = kp.kp[1]
                record[f'sk_{i}_z'] = kp.kp[2]
            
            skeleton_records.append(record)
else:
    # Manual parsing fallback - this is complex and depends on message structure
    print("ERROR: Manual skeleton parsing not yet implemented")
    print("Please ensure zed_msgs is available in the Docker environment")
    raise NotImplementedError("zed_msgs required for skeleton extraction")

df_skeletons = pd.DataFrame(skeleton_records)
print(f"\nSkeleton DataFrame shape: {df_skeletons.shape}")
print(f"Unique skeleton IDs: {df_skeletons['skeleton_id'].unique()}")
print(f"Time range: {df_skeletons['timestamp'].min():.2f}s to {df_skeletons['timestamp'].max():.2f}s")
print(f"Average frequency: {len(df_skeletons) / (df_skeletons['timestamp'].max() - df_skeletons['timestamp'].min()):.1f} Hz")

df_skeletons.head()

## 6. Interactive Skeleton Selection with Visualization

Visualize both detected skeletons with the robot to allow selection of the "leader".

In [ ]:
# Separate skeletons by ID
unique_skeleton_ids = sorted(df_skeletons['skeleton_id'].unique())
print(f"Detected skeleton IDs: {unique_skeleton_ids}")

# Get a sample frame (middle of recording) for visualization
mid_time = (df_skeletons['timestamp'].min() + df_skeletons['timestamp'].max()) / 2
sample_frame = df_skeletons.iloc[(df_skeletons['timestamp'] - mid_time).abs().argsort()[:2]]

print(f"\nSample frame for visualization (t={mid_time:.2f}s):")
print(sample_frame[['timestamp', 'skeleton_id', 'sk_0_x', 'sk_0_y', 'sk_0_z']].head())

In [ ]:
def plot_skeleton_and_robot(
    skeleton_df: pd.DataFrame,
    robot_joint_angles: Dict[str, float],
    fk: UR10ForwardKinematics,
    title: str = "Skeleton Selection"
) -> go.Figure:
    """
    Create 3D plot showing skeletons and robot for selection.
    
    Args:
        skeleton_df: DataFrame with skeleton data (can have multiple skeleton_ids)
        robot_joint_angles: Dictionary of robot joint angles
        fk: Forward kinematics instance
        title: Plot title
    
    Returns:
        Plotly figure
    """
    fig = go.Figure()
    
    # Plot robot
    robot_positions = np.array(fk.get_joint_positions(robot_joint_angles))
    fig.add_trace(go.Scatter3d(
        x=robot_positions[:, 0],
        y=robot_positions[:, 1],
        z=robot_positions[:, 2],
        mode='lines+markers',
        line=dict(color='gray', width=8),
        marker=dict(size=8, color='gray'),
        name='UR10 Robot',
        showlegend=True
    ))
    
    # Plot each skeleton
    colors = ['red', 'blue', 'green', 'orange']  # For up to 4 skeletons
    
    for skeleton_id in skeleton_df['skeleton_id'].unique():
        skel_row = skeleton_df[skeleton_df['skeleton_id'] == skeleton_id].iloc[0]
        
        # Extract keypoint positions
        keypoints = []
        for i in range(16):
            x = skel_row[f'sk_{i}_x']
            y = skel_row[f'sk_{i}_y']
            z = skel_row[f'sk_{i}_z']
            keypoints.append([x, y, z])
        
        keypoints = np.array(keypoints)
        
        # Plot keypoints
        color_idx = int(skeleton_id) % len(colors)
        fig.add_trace(go.Scatter3d(
            x=keypoints[:, 0],
            y=keypoints[:, 1],
            z=keypoints[:, 2],
            mode='markers',
            marker=dict(size=6, color=colors[color_idx]),
            name=f'Skeleton {skeleton_id}',
            showlegend=True
        ))
        
        # Define skeleton connections (ZED 16-point mode)
        connections = [
            (0, 1), (1, 2), (2, 3),  # Spine
            (3, 4), (4, 5), (5, 6), (6, 7),  # Left arm
            (3, 8), (8, 9), (9, 10), (10, 11),  # Right arm
            (0, 12), (12, 13),  # Left leg
            (0, 14), (14, 15)  # Right leg
        ]
        
        # Plot connections
        for start, end in connections:
            fig.add_trace(go.Scatter3d(
                x=[keypoints[start, 0], keypoints[end, 0]],
                y=[keypoints[start, 1], keypoints[end, 1]],
                z=[keypoints[start, 2], keypoints[end, 2]],
                mode='lines',
                line=dict(color=colors[color_idx], width=3),
                showlegend=False
            ))
    
    # Set layout
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            aspectmode='data'
        ),
        width=900,
        height=700
    )
    
    return fig


# Get robot joint angles at sample time
sample_joints = df_joints.iloc[(df_joints['timestamp'] - mid_time).abs().argsort()[0]]
sample_joint_angles = {
    fk.joint_names[i]: sample_joints[f'j{i}']
    for i in range(6)
}

# Create visualization
fig = plot_skeleton_and_robot(sample_frame, sample_joint_angles, fk, 
                              f"Skeleton Selection (t={mid_time:.2f}s)")
fig.show()

In [ ]:
# =============================================================================
# MANUAL SELECTION - Which skeleton is the leader?
# =============================================================================

SELECTED_SKELETON_ID = 0  # <-- CHANGE THIS AFTER VIEWING VISUALIZATION

# =============================================================================

print(f"Selected skeleton ID: {SELECTED_SKELETON_ID}")

# Filter to selected skeleton only
df_skeleton_selected = df_skeletons[df_skeletons['skeleton_id'] == SELECTED_SKELETON_ID].copy()
df_skeleton_selected = df_skeleton_selected.drop(columns=['skeleton_id'])
df_skeleton_selected = df_skeleton_selected.reset_index(drop=True)

print(f"\nSelected skeleton data shape: {df_skeleton_selected.shape}")
print(f"Time range: {df_skeleton_selected['timestamp'].min():.2f}s to {df_skeleton_selected['timestamp'].max():.2f}s")

# Validate selection exists throughout recording
time_gaps = df_skeleton_selected['timestamp'].diff()
max_gap = time_gaps.max()
print(f"\nMax time gap in selected skeleton: {max_gap:.3f}s")
if max_gap > 0.5:
    print("WARNING: Large gaps detected - skeleton tracking may be inconsistent")

## 7. Synchronize Skeleton and Joint Data

Interpolate joint states to skeleton timestamps.

In [ ]:
def synchronize_data(
    df_skeleton: pd.DataFrame,
    df_joints: pd.DataFrame,
    tolerance_sec: float = 0.05
) -> pd.DataFrame:
    """
    Synchronize skeleton and joint data by interpolating joints to skeleton timestamps.
    
    Args:
        df_skeleton: Skeleton DataFrame with timestamp column
        df_joints: Joint DataFrame with timestamp column
        tolerance_sec: Maximum time difference for synchronization
    
    Returns:
        Synchronized DataFrame with both skeleton and joint data
    """
    # Interpolate each joint column to skeleton timestamps
    joint_cols = [f'j{i}' for i in range(6)]
    
    interpolated_joints = {}
    for col in joint_cols:
        interpolated_joints[col] = np.interp(
            df_skeleton['timestamp'],
            df_joints['timestamp'],
            df_joints[col]
        )
    
    # Combine skeleton and interpolated joints
    df_synced = df_skeleton.copy()
    for col, values in interpolated_joints.items():
        df_synced[col] = values
    
    return df_synced


print("Synchronizing skeleton and joint data...")
df_synced = synchronize_data(df_skeleton_selected, df_joints)

print(f"\nSynchronized data shape: {df_synced.shape}")
print(f"Columns: {list(df_synced.columns)}")
print(f"\nSample rows:")
print(df_synced[['timestamp', 'sk_0_x', 'sk_0_y', 'sk_0_z', 'j0', 'j1', 'j2']].head())

## 8. Transform Skeleton to Robot Frame

Apply static transform from camera frame to robot_base_link frame.

**Note:** This requires the calibrated transform. For now, we'll use a placeholder.
In production, load from `config/camera_to_robot_transform.yaml`.

In [ ]:
# Load static transform (if available)
# For now, use identity transform as placeholder
# TODO: Load from calibration file

def apply_transform_to_skeleton(
    df: pd.DataFrame,
    translation: np.ndarray,
    rotation_matrix: np.ndarray
) -> pd.DataFrame:
    """
    Apply rigid transform to all skeleton keypoints.
    
    Args:
        df: DataFrame with skeleton keypoints (sk_i_x, sk_i_y, sk_i_z)
        translation: 3D translation vector
        rotation_matrix: 3x3 rotation matrix
    
    Returns:
        DataFrame with transformed skeleton keypoints
    """
    df_transformed = df.copy()
    
    for i in range(16):
        # Extract keypoint
        x = df[f'sk_{i}_x'].values
        y = df[f'sk_{i}_y'].values
        z = df[f'sk_{i}_z'].values
        
        # Apply rotation and translation
        keypoints = np.stack([x, y, z], axis=1)  # [N, 3]
        keypoints_transformed = (rotation_matrix @ keypoints.T).T + translation
        
        # Update DataFrame
        df_transformed[f'sk_{i}_x'] = keypoints_transformed[:, 0]
        df_transformed[f'sk_{i}_y'] = keypoints_transformed[:, 1]
        df_transformed[f'sk_{i}_z'] = keypoints_transformed[:, 2]
    
    return df_transformed


# Placeholder transform (identity)
# TODO: Replace with actual calibrated transform
camera_to_robot_translation = np.array([0.0, 0.0, 0.0])
camera_to_robot_rotation = np.eye(3)

print("Applying coordinate transform to skeleton...")
print("WARNING: Using identity transform (placeholder)")
print("TODO: Load calibrated transform from config")

df_synced_transformed = apply_transform_to_skeleton(
    df_synced,
    camera_to_robot_translation,
    camera_to_robot_rotation
)

print("\nTransform applied successfully")

## 9. Quality Checks

In [ ]:
def quality_checks(df: pd.DataFrame, fk: UR10ForwardKinematics) -> Dict[str, Any]:
    """
    Perform quality checks on synchronized data.
    
    Returns:
        Dictionary with quality metrics
    """
    metrics = {}
    
    # 1. Check for NaN values
    metrics['nan_count'] = df.isna().sum().sum()
    
    # 2. Check for position jumps in skeleton
    pelvis_positions = df[['sk_0_x', 'sk_0_y', 'sk_0_z']].values
    pelvis_velocity = np.linalg.norm(np.diff(pelvis_positions, axis=0), axis=1)
    time_diffs = np.diff(df['timestamp'].values)
    pelvis_speed = pelvis_velocity / time_diffs
    
    metrics['max_pelvis_speed'] = pelvis_speed.max()
    metrics['mean_pelvis_speed'] = pelvis_speed.mean()
    metrics['jumps_over_2ms'] = (pelvis_speed > 2.0).sum()  # 2 m/s threshold
    
    # 3. Check joint limits
    joint_limit_violations = 0
    for i, joint_name in enumerate(fk.joint_names):
        limits = fk.joint_info[joint_name]['limits']
        joint_values = df[f'j{i}'].values
        violations = ((joint_values < limits[0]) | (joint_values > limits[1])).sum()
        joint_limit_violations += violations
    
    metrics['joint_limit_violations'] = joint_limit_violations
    
    # 4. Check temporal consistency
    metrics['num_samples'] = len(df)
    metrics['duration_sec'] = df['timestamp'].max() - df['timestamp'].min()
    metrics['avg_frequency_hz'] = metrics['num_samples'] / metrics['duration_sec']
    
    return metrics


print("Performing quality checks...\n")
quality_metrics = quality_checks(df_synced_transformed, fk)

print("Quality Metrics:")
print("=" * 50)
for key, value in quality_metrics.items():
    print(f"{key:30s}: {value}")
print("=" * 50)

# Warnings
if quality_metrics['nan_count'] > 0:
    print(f"\nWARNING: {quality_metrics['nan_count']} NaN values detected")

if quality_metrics['jumps_over_2ms'] > 0:
    print(f"\nWARNING: {quality_metrics['jumps_over_2ms']} large position jumps detected")

if quality_metrics['joint_limit_violations'] > 0:
    print(f"\nWARNING: {quality_metrics['joint_limit_violations']} joint limit violations detected")

## 10. Export to CSV

In [ ]:
# Determine next recording ID
if metadata_file.exists():
    df_metadata = pd.read_csv(metadata_file)
    next_rec_id = df_metadata['recording_id'].max() + 1
else:
    next_rec_id = 1
    df_metadata = pd.DataFrame()

rec_name = f"rec_{next_rec_id:03d}"
csv_output_path = csv_output_dir / f"{rec_name}.csv"

print(f"Recording ID: {next_rec_id}")
print(f"Output CSV: {csv_output_path}")

# Reorder columns for CSV export
skeleton_cols = [f'sk_{i}_{axis}' for i in range(16) for axis in ['x', 'y', 'z']]
joint_cols = [f'j{i}' for i in range(6)]
export_cols = ['timestamp'] + skeleton_cols + joint_cols

df_export = df_synced_transformed[export_cols]

# Save CSV
df_export.to_csv(csv_output_path, index=False)
print(f"\nExported {len(df_export)} rows to {csv_output_path}")
print(f"CSV shape: {df_export.shape}")
print(f"CSV size: {csv_output_path.stat().st_size / 1024 / 1024:.2f} MB")

## 11. Update Metadata

In [ ]:
# Create metadata entry
metadata_entry = {
    'recording_id': next_rec_id,
    'csv_filename': f"{rec_name}.csv",
    'rosbag_name': ROSBAG_NAME,
    'selected_skeleton_id': SELECTED_SKELETON_ID,
    'num_samples': len(df_export),
    'duration_sec': quality_metrics['duration_sec'],
    'avg_frequency_hz': quality_metrics['avg_frequency_hz'],
    'processed_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'quality_nan_count': quality_metrics['nan_count'],
    'quality_jumps': quality_metrics['jumps_over_2ms'],
    'quality_joint_violations': quality_metrics['joint_limit_violations']
}

# Append to metadata file
df_metadata_new = pd.DataFrame([metadata_entry])
if metadata_file.exists():
    df_metadata = pd.concat([df_metadata, df_metadata_new], ignore_index=True)
else:
    df_metadata = df_metadata_new

df_metadata.to_csv(metadata_file, index=False)
print(f"\nUpdated metadata file: {metadata_file}")
print(f"\nMetadata entry:")
print(df_metadata_new.T)

## 12. Summary

In [ ]:
print("=" * 70)
print("PHASE 2 PROCESSING COMPLETE")
print("=" * 70)

print(f"\nRosbag: {ROSBAG_NAME}")
print(f"Recording ID: {next_rec_id}")
print(f"Selected Skeleton: {SELECTED_SKELETON_ID}")
print(f"\nOutput:")
print(f"  CSV: {csv_output_path}")
print(f"  Rows: {len(df_export)}")
print(f"  Columns: {len(df_export.columns)}")
print(f"  Duration: {quality_metrics['duration_sec']:.2f}s")
print(f"  Frequency: {quality_metrics['avg_frequency_hz']:.1f} Hz")

print(f"\nQuality:")
print(f"  NaN values: {quality_metrics['nan_count']}")
print(f"  Position jumps: {quality_metrics['jumps_over_2ms']}")
print(f"  Joint violations: {quality_metrics['joint_limit_violations']}")

print("\n" + "=" * 70)
print("NEXT STEPS")
print("=" * 70)
print("""
1. Change ROSBAG_NAME at the top of this notebook to process the next rosbag
2. Run the notebook again to process additional recordings
3. Once all rosbags are processed, proceed to:
   - 02_prepare_training_data.ipynb: Create PyTorch datasets
   - 03_train_vam.ipynb: Train the Vision-Action Model
""")

print("=" * 70)